In [61]:
from uuid import UUID

from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient
from pydantic_ai import Agent, RunContext
from pydantic_ai import messages as _messages

from knd.mem_functions import create_agent_experience, create_user_specific_experience
from knd.mem_models import Agent as AgentDocument
from knd.mem_models import Memory, Profile, Task, User

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
client = AsyncIOMotorClient("mongodb://localhost:27017")

In [63]:
await client.list_database_names()

['admin', 'agent_db', 'config', 'local', 'test_db']

In [64]:
await client.drop_database("agent_db")

In [65]:
await init_beanie(database=client.agent_db, document_models=[User, AgentDocument, Task])

In [6]:
profile = Profile(name="hamza", interests=["football", "python", "ai"])
memories = [
    Memory(
        id=UUID("1ea351e0-5920-47a0-a358-1cc3f1fdda0d"),
        context="last anime episode",
        category="fact",
        content="watched solo leveling season 2 ep 6. was cool",
    ),
    Memory(context="last watched football match", category="fact", content="barcelona 2-1 real madrid"),
]

In [7]:
user = User(profile=profile, memories=memories)

In [8]:
agent_doc = AgentDocument(
    name="joke_teller",
    model="google-gla:gemini-1.5-flash",
    system_prompt="You are a joke teller. talk like tony stark",
)

In [9]:
user = await user.insert()
agent_doc = await agent_doc.insert()

In [ ]:
task = Task(user=user, agent=agent_doc) # type: ignore
task = await task.insert()

In [12]:
# message_history = [
#     _messages.ModelRequest(parts=[_messages.UserPromptPart(content="tell me a joke")]),
#     _messages.ModelResponse(
#         parts=[
#             _messages.TextPart(
#                 content="Hey there! *adjusts sunglasses* Why don't scientists trust atoms? Because they make up everything! *smirks* Get it? Because atoms literally make up everything in the universe, and also 'make up' as in... ah, you got it. Pure genius, if I do say so myself. JARVIS, add that one to my collection of greatest hits."
#             )
#         ]
#     ),
#     _messages.ModelRequest(
#         parts=[_messages.UserPromptPart(content="Can you make your jokes shorter? Just one-liners please.")]
#     ),
#     _messages.ModelResponse(
#         parts=[
#             _messages.TextPart(
#                 content="*Flashes signature smirk* What do you call a fake noodle? An impasta! JARVIS, that's what I call efficiency."
#             )
#         ]
#     ),
# ]


In [13]:
joke_teller = Agent(
    name=agent_doc.name, model=agent_doc.model, system_prompt=agent_doc.system_prompt, deps_type=Task
)


@joke_teller.system_prompt(dynamic=True)
def system_prompt(ctx: RunContext[Task]) -> str:
    return ctx.deps.experience_str()

In [ ]:
user_prompt = "tell me a joke"
message_history = None
while user_prompt.lower() != "q":
    res = await joke_teller.run(user_prompt=user_prompt, message_history=message_history, deps=task)
    user_prompt = input(f"{res.data} > ")
    message_history = res.all_messages()
    for msg in res.new_messages():
        task.add_message(content=msg)
    await task.save()

/tmp/ipykernel_378576/3530239306.py:4: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  res = await joke_teller.run(user_prompt=user_prompt, message_history=message_history, deps=task)


In [43]:
memory_agent = Agent(model="google-gla:gemini-1.5-flash", name="memory_agent")

In [ ]:
generated_user = await create_user_specific_experience(
    memory_agent=memory_agent, message_history=task.message_history
)

/home/hamza/dev/knd/.venv/lib/python3.12/site-packages/pydantic/type_adapter.py:527: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `ModelRequest` but got `dict` with value `{'parts': [{'content': 'Y...t'}], 'kind': 'request'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelResponse` but got `dict` with value `{'parts': [{'content': 'Y...t'}], 'kind': 'request'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelRequest` but got `dict` with value `{'parts': [{'content': "A...00), 'kind': 'response'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelResponse` but got `dict` with value `{'parts': [{'content': "A...00), 'kind': 'response'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelRequest` but got `dict` with value `{'parts': [{'content': 'm...t'}

[ModelRequest(parts=[SystemPromptPart(content='You are a joke teller. talk like tony stark', dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='<agent_experience>\n{"procedural_knowledge":"","common_scenarios":[],"effective_strategies":[],"known_pitfalls":[],"tool_patterns":[],"heuristics":[],"user_feedback":[],"improvement_areas":[]}\n</agent_experience>\n\n<user_specific_experience>\n<user_profile>\n{"name":"hamza","age":null,"interests":["football","python","ai"],"home":"","occupation":"","conversation_preferences":[]}\n</user_profile>\n\n<memories>\n{"id":"1ea351e0-5920-47a0-a358-1cc3f1fdda0d","created_at":"2025-02-19T14:52:24.606314","context":"last anime episode","category":"fact","content":"watched solo leveling season 2 ep 6. was cool"}\n{"id":"dc0cf27e-2317-4565-b8a7-669346ac97d3","created_at":"2025-02-19T14:52:24.606354","context":"last watched football match","category":"fact","content":"barcelona 2-1 real madrid"}\n</memories>\n</user_specific_experienc

In [20]:
generated_user.model_dump()

{'profile': {'name': 'hamza',
  'age': None,
  'interests': ['football', 'python', 'ai', 'jokes'],
  'home': '',
  'occupation': '',
  'conversation_preferences': []},
 'memories': [{'id': UUID('49b18029-5d14-436d-ac58-0cd314e2f75d'),
   'created_at': datetime.datetime(2025, 2, 19, 14, 54, 48, 654080),
   'context': 'Use this to tailor future jokes and conversational tone.  Avoid long, drawn-out explanations or overly serious topics unless explicitly requested.',
   'category': 'Interaction Patterns',
   'content': 'User appreciates short, one-liner jokes.  Prefers a conversational style that is witty and self-deprecating (like Tony Stark).',
   'superseded_ids': []}]}

In [49]:
user.update_from_generated_user(generated_user)

In [50]:
user.model_dump()

{'id': ObjectId('67b5a9d88ff544c8cb2211a9'),
 'profile': {'name': 'hamza',
  'age': None,
  'interests': ['football', 'python', 'ai', 'jokes'],
  'home': '',
  'occupation': '',
  'conversation_preferences': []},
 'memories': [{'id': UUID('1ea351e0-5920-47a0-a358-1cc3f1fdda0d'),
   'created_at': datetime.datetime(2025, 2, 19, 14, 52, 24, 606000),
   'context': 'last anime episode',
   'category': 'fact',
   'content': 'watched solo leveling season 2 ep 6. was cool',
   'superseded_ids': []},
  {'id': UUID('dc0cf27e-2317-4565-b8a7-669346ac97d3'),
   'created_at': datetime.datetime(2025, 2, 19, 14, 52, 24, 606000),
   'context': 'last watched football match',
   'category': 'fact',
   'content': 'barcelona 2-1 real madrid',
   'superseded_ids': []},
  {'id': UUID('49b18029-5d14-436d-ac58-0cd314e2f75d'),
   'created_at': datetime.datetime(2025, 2, 19, 14, 54, 48, 654000),
   'context': 'Use this to tailor future jokes and conversational tone.  Avoid long, drawn-out explanations or overly 

In [23]:
user = await user.save()

In [39]:
user.profile.age = 29
user = await user.save()

In [24]:
agent_experience = await create_agent_experience(memory_agent=memory_agent, message_history=task.message_history)

/home/hamza/dev/knd/.venv/lib/python3.12/site-packages/pydantic/type_adapter.py:527: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue: Expected `ModelRequest` but got `dict` with value `{'parts': [{'content': 'Y...t'}], 'kind': 'request'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelResponse` but got `dict` with value `{'parts': [{'content': 'Y...t'}], 'kind': 'request'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelRequest` but got `dict` with value `{'parts': [{'content': "A...00), 'kind': 'response'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelResponse` but got `dict` with value `{'parts': [{'content': "A...00), 'kind': 'response'}` - serialized value may not be as expected
  PydanticSerializationUnexpectedValue: Expected `ModelRequest` but got `dict` with value `{'parts': [{'content': 'm...t'}

In [25]:
agent_experience.model_dump()

{'procedural_knowledge': '',
 'common_scenarios': ['"Tell me a joke"',
  '"Tell me a short joke"',
  '"One liner joke"'],
 'effective_strategies': ['Adjusting joke length based on user preference',
  'Using humor styles appropriate for general audiences'],
 'known_pitfalls': [],
 'tool_patterns': [],
 'heuristics': ['Shorter jokes are generally preferred for quick interactions',
  'One-liner jokes are efficient for brevity'],
 'user_feedback': [],
 'improvement_areas': ['Expanding joke repertoire across various humor styles',
  'Developing mechanisms for user preference learning and personalized humor delivery']}

In [27]:
if agent_experience:
    agent_doc.experience = agent_experience
    agent_doc = await agent_doc.save()

In [29]:
agent_doc.model_dump()

{'id': ObjectId('67b5a9d88ff544c8cb2211aa'),
 'name': 'joke_teller',
 'model': 'google-gla:gemini-1.5-flash',
 'description': '',
 'system_prompt': 'You are a joke teller. talk like tony stark',
 'experience': {'procedural_knowledge': '',
  'common_scenarios': ['"Tell me a joke"',
   '"Tell me a short joke"',
   '"One liner joke"'],
  'effective_strategies': ['Adjusting joke length based on user preference',
   'Using humor styles appropriate for general audiences'],
  'known_pitfalls': [],
  'tool_patterns': [],
  'heuristics': ['Shorter jokes are generally preferred for quick interactions',
   'One-liner jokes are efficient for brevity'],
  'user_feedback': [],
  'improvement_areas': ['Expanding joke repertoire across various humor styles',
   'Developing mechanisms for user preference learning and personalized humor delivery']}}

In [41]:
task = await task.save()

In [42]:
print(task.experience_str())

<agent_experience>
{"procedural_knowledge":"","common_scenarios":["\"Tell me a joke\"","\"Tell me a short joke\"","\"One liner joke\""],"effective_strategies":["Adjusting joke length based on user preference","Using humor styles appropriate for general audiences"],"known_pitfalls":[],"tool_patterns":[],"heuristics":["Shorter jokes are generally preferred for quick interactions","One-liner jokes are efficient for brevity"],"user_feedback":[],"improvement_areas":["Expanding joke repertoire across various humor styles","Developing mechanisms for user preference learning and personalized humor delivery"]}
</agent_experience>

<user_specific_experience>
<user_profile>
{"name":"hamza","age":29,"interests":["football","python","ai","jokes"],"home":"","occupation":"","conversation_preferences":[]}
</user_profile>

<memories>
{"id":"1ea351e0-5920-47a0-a358-1cc3f1fdda0d","created_at":"2025-02-19T14:52:24.606000","context":"last anime episode","category":"fact","content":"watched solo leveling se

In [66]:
await user.save()
await agent_doc.save()

Agent(id=ObjectId('67b5a9d88ff544c8cb2211aa'), revision_id=None, name='joke_teller', model='google-gla:gemini-1.5-flash', description='', system_prompt='You are a joke teller. talk like tony stark', experience=AgentExperience(procedural_knowledge='', common_scenarios=['"Tell me a joke"', '"Tell me a short joke"', '"One liner joke"'], effective_strategies=['Adjusting joke length based on user preference', 'Using humor styles appropriate for general audiences'], known_pitfalls=[], tool_patterns=[], heuristics=['Shorter jokes are generally preferred for quick interactions', 'One-liner jokes are efficient for brevity'], user_feedback=[], improvement_areas=['Expanding joke repertoire across various humor styles', 'Developing mechanisms for user preference learning and personalized humor delivery']))

In [67]:
task2 = Task(user=user, agent=agent_doc)

In [68]:
user_prompt = "tell me a joke"
message_history = None
while user_prompt.lower() != "q":
    res = await joke_teller.run(user_prompt=user_prompt, message_history=message_history, deps=task2)
    user_prompt = input(f"{res.data} > ")
    message_history = res.all_messages()
    for msg in res.new_messages():
        task2.add_message(content=msg)
    await task2.save()

In [69]:
task2.message_history

[{'parts': [{'content': 'You are a joke teller. talk like tony stark',
    'dynamic_ref': None,
    'part_kind': 'system-prompt'},
   {'content': '<agent_experience>\n{"procedural_knowledge":"","common_scenarios":["\\"Tell me a joke\\"","\\"Tell me a short joke\\"","\\"One liner joke\\""],"effective_strategies":["Adjusting joke length based on user preference","Using humor styles appropriate for general audiences"],"known_pitfalls":[],"tool_patterns":[],"heuristics":["Shorter jokes are generally preferred for quick interactions","One-liner jokes are efficient for brevity"],"user_feedback":[],"improvement_areas":["Expanding joke repertoire across various humor styles","Developing mechanisms for user preference learning and personalized humor delivery"]}\n</agent_experience>\n\n<user_specific_experience>\n<user_profile>\n{"name":"hamza","age":null,"interests":["football","python","ai","jokes"],"home":"","occupation":"","conversation_preferences":[]}\n</user_profile>\n\n<memories>\n{"id":